In [4]:
#Importamos librerías
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import unicodedata
import ftfy
import difflib
from scipy.stats import ks_2samp, mannwhitneyu, chi2_contingency
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.ensemble import RandomForestClassifier

In [5]:
#Cargamos los dataset
inv = pd.read_csv("inventario_central_v2.csv")
trx = pd.read_csv("transacciones_logistica_v2.csv")
fb  = pd.read_csv("feedback_clientes_v2.csv")


In [ ]:
# Función para auditar los dataset
def audit(df):
    return pd.DataFrame({
        "nulos_%": df.isna().mean() * 100,
        "tipo": df.dtypes
    })

## Auditoría de Calidad y Transparencia — Dataset Inventario (inv)


In [ ]:
display(inv.head())

,SKU_ID,Categoria,Stock_Actual,Costo_Unitario_USD,Punto_Reorden,Lead_Time_Dias,Bodega_Origen,Ultima_Revision
0,PROD-1000,smart-phone,NaN,870.38,259,25-30 días,Norte,2025-11-17
1,PROD-1001,Accesorios,476.0,1397.26,169,25-30 días,Norte,2024-03-05
2,PROD-1002,Monitores,1209.0,611.62,214,5,Sur,2024-06-21
3,PROD-1003,smart-phone,1825.0,145.94,187,10,Sur,2025-01-07
4,PROD-1004,Smartphones,1713.0,77.78,105,5,Sur,2024-07-04


In [ ]:
audit(inv)

,nulos_%,tipo
SKU_ID,0.00,object
Categoria,0.00,object
Stock_Actual,4.00,float64
Costo_Unitario_USD,0.00,float64
Punto_Reorden,0.00,int64
Lead_Time_Dias,16.12,object
Bodega_Origen,0.00,object
Ultima_Revision,0.00,object


### FUNCIONES DE LIMPIEZA Y NORMALIZACIÓN

In [ ]:
# Función realizada por IA
def parse_lead_time(x):
    """
    Convierte Lead_Time_Dias a numérico:
    - 'Inmediato' -> 1
    - '25-30 días' -> 30 (toma el valor mayor del rango)
    - '3', '5', '10' -> float
    - NaN se mantiene como NaN
    """
    if pd.isna(x):
        return np.nan

    s = str(x).strip().lower()

    if s in {"inmediato", "inmediate", "immediate"}:
        return 1.0

    # Rango tipo "25-30 días" o "25 - 30"
    m = re.search(r"(\d+(?:\.\d+)?)\s*-\s*(\d+(?:\.\d+)?)", s)
    if m:
        a, b = float(m.group(1)), float(m.group(2))
        return max(a, b)

    # Primer número que aparezca
    m = re.search(r"(\d+(?:\.\d+)?)", s)
    if m:
        return float(m.group(1))

    return np.nan


In [ ]:
# Función realizada por IA para estandarizar la columna categoría
def parse_categoria(x):
    if pd.isna(x):
        return np.nan

    s = str(x).strip().lower()

    # Normalizar separadores
    s = re.sub(r"[_\-\/]+", " ", s)

    # Quitar caracteres no alfanuméricos
    s = re.sub(r"[^a-z0-9\s]", "", s)

    # Colapsar espacios
    s = re.sub(r"\s+", " ", s).strip()

    # Si quedó vacío (ej. "???", "---", "***") → NaN
    if s == "":
        return np.nan

    # Canonicalización semántica
    if "smart" in s and "phone" in s:
        return "Smartphones"

    if s in {"laptop", "laptops"}:
        return "Laptops"

    return s.title()



In [ ]:

## Función realiza por IA para verifcar si hay más de dos filas con datos nulos dados que no se podría imputar
def drop_rows_with_many_nulls(df, k=2):
    """
    Elimina filas que tengan k o más valores nulos.
    Retorna:
      - df_limpio
      - df_eliminados (con las columnas que causaron la eliminación)
    """
    # Contar nulos por fila
    null_count = df.isna().sum(axis=1)

    # Filas a eliminar
    mask_drop = null_count >= k

    # Identificar qué columnas están nulas en esas filas
    cols_nulas = (
        df[mask_drop]
        .isna()
        .apply(lambda row: list(df.columns[row]), axis=1)
    )

    df_eliminados = df[mask_drop].copy()
    df_eliminados["Columnas_Nulas"] = cols_nulas
    df_eliminados["N_Nulos"] = null_count[mask_drop]

    # Dataset limpio
    df_limpio = df[~mask_drop].copy()

    return df_limpio, df_eliminados


### NORMALIZACION DE CAMPOS (Lead_Time_Dias, Categoria)

Dentro de la columna Lead_Time_Dias, se detectan valores atípicos de carácter cualitativo, tales como "inmediato" o intervalos de tiempo que oscilan entre "25–30 días". En el primer escenario, el término "inmediato" se sustituye por 1, lo que sugiere que el producto puede ser despachado el mismo día; en el segundo escenario, se preserva el valor superior del rango para integrar un margen de seguridad en la planificación logística.

Además, la columna Categoría se estandariza para prevenir inconsistencias en la nomenclatura que impacten los análisis y las visualizaciones, unificando variantes como "smart-phone", "smart phone" o "Smartphone" bajo una única etiqueta coherente.

In [ ]:
inv["Lead_Time_Dias"] = inv["Lead_Time_Dias"].apply(parse_lead_time)
inv["Categoria"] = inv["Categoria"].apply(parse_categoria)


In [ ]:
# Validación
display(inv[["Lead_Time_Dias", "Categoria"]].head())
print("Categorías únicas (muestra):", inv["Categoria"].dropna().unique()[:15])

,Lead_Time_Dias,Categoria
0,30.0,Smartphones
1,30.0,Accesorios
2,5.0,Monitores
3,10.0,Smartphones
4,5.0,Smartphones


Categorías únicas (muestra): ['Smartphones' 'Accesorios' 'Monitores' 'Tablets' 'Laptops']



### FILTRADO DE FILAS CON NULOS (k ≥ 2)



Para definir la estrategia de imputación, primero se identifican los registros que presentan más de dos valores nulos en sus columnas; estos registros se eliminan, ya que una imputación tan extensa introduciría un nivel de incertidumbre que podría distorsionar los análisis posteriores. De este modo, solo se imputan aquellos registros con faltantes limitados, donde la reconstrucción de la información es estadísticamente y operativamente confiable.

In [ ]:
inv_limpio, inv_eliminados = drop_rows_with_many_nulls(inv, k=2)
print("Filas eliminadas:", len(inv_eliminados))
print("Filas restantes:", len(inv_limpio))


Filas eliminadas: 87
Filas restantes: 2413


### IMPUTACION 1: Lead_Time_Dias

Ahora, imputamos los valores nulos de Lead_Time_Dia con la mediana

In [ ]:
mediana_lt = inv_limpio["Lead_Time_Dias"].median()
n_nulos_antes = inv_limpio["Lead_Time_Dias"].isna().sum()

inv_limpio["Lead_Time_Dias"] = inv_limpio["Lead_Time_Dias"].fillna(mediana_lt)

n_nulos_despues = inv_limpio["Lead_Time_Dias"].isna().sum()
valores_finales = np.sort(inv_limpio["Lead_Time_Dias"].unique())

print(f"Mediana usada: {mediana_lt}")
print(f"Valores imputados: {n_nulos_antes}")
print(f"Nulos restantes: {n_nulos_despues}")
print(f"Valores únicos finales (Lead_Time_Dias):\n{valores_finales}")

Mediana usada: 5.0
Valores imputados: 323
Nulos restantes: 0
Valores únicos finales (Lead_Time_Dias):
[ 1.  3.  5. 10. 30.]


### NORMALIZACION Y ANALISIS DE BODEGA_ORIGEN

In [ ]:
inv_limpio["Bodega_Origen"].unique()

array(['Norte', 'Sur', 'ZONA_FRANCA', 'norte', 'BOD-EXT-99', 'Occidente'],
      dtype=object)

In [ ]:
valores_bodega_origen = inv_limpio["Bodega_Origen"].dropna().unique()

print(f"Valores únicos en Bodega_Origen ({len(valores_bodega_origen)}):")
print(valores_bodega_origen)


Valores únicos en Bodega_Origen (6):
['Norte' 'Sur' 'ZONA_FRANCA' 'norte' 'BOD-EXT-99' 'Occidente']


In [ ]:
# Normalizar solo los valores específicos
inv_limpio["Bodega_Origen"] = inv_limpio["Bodega_Origen"].replace({
    "norte": "Norte",
    "ZONA_FRANCA": "Zona Franca"
})

valores_bodega_origen = np.sort(inv_limpio["Bodega_Origen"].dropna().unique())

print(f"Valores únicos en Bodega_Origen ({len(valores_bodega_origen)}):")
print(valores_bodega_origen)

Valores únicos en Bodega_Origen (5):
['BOD-EXT-99' 'Norte' 'Occidente' 'Sur' 'Zona Franca']


In [ ]:
df_lt = inv_limpio[inv_limpio["Lead_Time_Dias"].notna()].copy()

bod_ext = df_lt[df_lt["Bodega_Origen"] == "BOD-EXT-99"]
zonas = df_lt[df_lt["Bodega_Origen"].isin(["Norte", "Sur", "Occidente", "Zona Franca"])]

# Estadísticas por zona
stats_zonas = zonas.groupby("Bodega_Origen")["Lead_Time_Dias"].agg(
    n="count",
    media="mean",
    mediana="median",
    std="std"
)

# Estadísticas de BOD-EXT-99
stats_bod = bod_ext["Lead_Time_Dias"].agg(
    n="count",
    media="mean",
    mediana="median",
    std="std"
)

print("Estadísticas por zona")
print(stats_zonas)

print("\nEstadísticas BOD-EXT-99")
print(stats_bod)

# Comparar distancia de medianas
distancias = (stats_zonas["mediana"] - stats_bod["mediana"]).abs()

print("\nDistancia de medianas contra BOD-EXT-99:")
print(distancias)



Estadísticas por zona
                 n      media  mediana        std
Bodega_Origen                                    
Norte          813   9.460025      5.0  10.166953
Occidente      372  10.293011      5.0  10.758694
Sur            422   9.097156      5.0  10.107383
Zona Franca    394   9.076142      5.0   9.845850

Estadísticas BOD-EXT-99
n          412.000000
media        9.771845
mediana      5.000000
std         10.399859
Name: Lead_Time_Dias, dtype: float64

Distancia de medianas contra BOD-EXT-99:
Bodega_Origen
Norte          0.0
Occidente      0.0
Sur            0.0
Zona Franca    0.0
Name: mediana, dtype: float64


BOD-EXT-99 presenta un comportamiento similar al de una bodega promedio, pero al corresponder a una bodega externa, se mantiene como una entidad diferenciada. En consecuencia, se trata como una nueva bodega dentro del modelo de datos, preservando su identidad operativa sin mezclarla con las bodegas internas.

In [ ]:
# Normalizar BOD-EXT-99
inv_limpio["Bodega_Origen"] = inv_limpio["Bodega_Origen"].replace({
    "BOD-EXT-99": "Externa"
})


### IMPUTACION 2: Stock_Actual por mediana dentro de Categoria

La imputación de Stock_Actual se realiza de manera condicional por grupo y no mediante un valor global, ya que el comportamiento del inventario depende fuertemente de la categoría del producto. Por ello, los registros se agrupan por Categoría y se utiliza la mediana de Stock_Actual de cada grupo como valor de imputación, lo que permite preservar la estructura y escala operativa de cada categoría y evita distorsionar la distribución real del inventario.

In [ ]:
# Calcular mediana de Stock_Actual por Categoría
medianas_por_categoria = inv_limpio.groupby("Categoria")["Stock_Actual"].median()

# Contar cuántos valores nulos hay antes
n_nulos_antes = inv_limpio["Stock_Actual"].isna().sum()

# Imputar usando la mediana de su propia categoría
inv_limpio["Stock_Actual"] = inv_limpio.apply(
    lambda row: medianas_por_categoria[row["Categoria"]]
                if pd.isna(row["Stock_Actual"])
                else row["Stock_Actual"],
    axis=1
)

# Contar cuántos quedan después
n_nulos_despues = inv_limpio["Stock_Actual"].isna().sum()

# Mostrar resultados
print("Medianas por Categoría:")
print(medianas_por_categoria)

print(f"\nValores imputados: {n_nulos_antes}")
print(f"Nulos restantes después de imputar: {n_nulos_despues}")


Medianas por Categoría:
Categoria
Accesorios      977.5
Laptops         940.0
Monitores       940.5
Smartphones    1041.0
Tablets         986.0
Name: Stock_Actual, dtype: float64

Valores imputados: 72
Nulos restantes después de imputar: 0


In [ ]:
audit(inv_limpio)

,nulos_%,tipo
SKU_ID,0.000000,object
Categoria,9.697472,object
Stock_Actual,0.000000,float64
Costo_Unitario_USD,0.000000,float64
Punto_Reorden,0.000000,int64
Lead_Time_Dias,0.000000,float64
Bodega_Origen,0.000000,object
Ultima_Revision,0.000000,object


La categoría se imputa utilizando Bodega_Origen y Stock_Actual porque el análisis evidenció que el Lead Time no aporta capacidad de discriminación (la mediana es de 5 días en todas las combinaciones), mientras que el nivel de inventario sí distingue de manera consistente las categorías dentro de cada bodega. Por ello, para cada registro sin categoría se identifica, dentro de su misma bodega, la categoría cuya mediana de stock es más cercana a su Stock_Actual, asignando así la categoría que mejor se ajusta a su perfil operativo real. Este criterio preserva la lógica logística del inventario y evita introducir sesgos artificiales en la estructura de los datos.

### IMPUTACION 3: Categoria faltante usando perfil de stock por bodega

In [ ]:
nulos_antes = inv_limpio["Categoria"].isna().sum()
print("Nulos en Categoria (antes):", nulos_antes)

Nulos en Categoria (antes): 234


In [ ]:
perfil = inv_limpio.dropna(subset=["Categoria"]).groupby(
    ["Bodega_Origen", "Categoria"]
).agg(
    stock_mediana=("Stock_Actual", "median"),
    n=("Categoria", "count")
).reset_index()


In [ ]:
display(perfil.head(10))

,Bodega_Origen,Categoria,stock_mediana,n
0,Externa,Accesorios,977.5,50
1,Externa,Laptops,954.0,92
2,Externa,Monitores,940.5,63
3,Externa,Smartphones,1041.0,115
4,Externa,Tablets,980.0,47
5,Norte,Accesorios,1072.5,118
6,Norte,Laptops,942.0,221
7,Norte,Monitores,920.0,116
8,Norte,Smartphones,1041.0,188
9,Norte,Tablets,955.0,96


In [ ]:
def inferir_categoria_por_stock(row):
    zona = row["Bodega_Origen"]
    stock = row["Stock_Actual"]

    sub = perfil[perfil["Bodega_Origen"] == zona]

    # Si la bodega no tiene suficiente info, usar todas
    if len(sub) == 0:
        sub = perfil

    distancias = (sub["stock_mediana"] - stock).abs()

    return sub.loc[distancias.idxmin(), "Categoria"]


In [ ]:
nulos_antes = inv_limpio["Categoria"].isna().sum()

inv_limpio.loc[inv_limpio["Categoria"].isna(), "Categoria"] = (
    inv_limpio[inv_limpio["Categoria"].isna()].apply(inferir_categoria_por_stock, axis=1)
)

nulos_despues = inv_limpio["Categoria"].isna().sum()

print(f"Categorías imputadas: {nulos_antes}")
print(f"Nulos restantes: {nulos_despues}")


Categorías imputadas: 234
Nulos restantes: 0


In [ ]:
audit(inv_limpio)

,nulos_%,tipo
SKU_ID,0.0,object
Categoria,0.0,object
Stock_Actual,0.0,float64
Costo_Unitario_USD,0.0,float64
Punto_Reorden,0.0,int64
Lead_Time_Dias,0.0,float64
Bodega_Origen,0.0,object
Ultima_Revision,0.0,object


### CORRECIÓN DATOS DE CODIFICACION

In [ ]:
# Filas con stock negativo
stock_negativo = inv_limpio[inv_limpio["Stock_Actual"] < 0]

print("Registros con Stock negativo")
print(f"Cantidad: {len(stock_negativo)}")
display(stock_negativo.sort_values("Stock_Actual"))

# Filas con costo extremadamente bajo (ej: 0.05 USD)
precios_muy_bajos = inv_limpio[inv_limpio["Costo_Unitario_USD"] < 1]

print("\nRegistros con Costo_Unitario_USD < 1")
print(f"Cantidad: {len(precios_muy_bajos)}")
display(precios_muy_bajos.sort_values("Costo_Unitario_USD"))

# Filas con costo extremadamente alto (ej: 850000 USD)
precios_muy_altos = inv_limpio[inv_limpio["Costo_Unitario_USD"] > 10000]

print("\nRegistros con Costo_Unitario_USD > 10,000")
print(f"Cantidad: {len(precios_muy_altos)}")
display(precios_muy_altos.sort_values("Costo_Unitario_USD", ascending=False))

Los valores negativos de Stock_Actual, aunque parezcan “ilógicos” físicamente, son aceptados contablemente en muchos ERPs. Estos valores muestran inventario reservado por ventas que se registraron antes de recibir los productos, pedidos pendientes, ajustes retroactivos, errores de sincronización entre almacenes o conteos físicos realizados después. Además, hay 57 casos distribuidos en diferentes categorías, bodegas y fechas, lo que indica un patrón de funcionamiento del sistema y no solo errores de escritura con un “-”. Por eso se mantienen igual en el conjunto de datos transformado 

In [ ]:
# Smartphone con 0.05 USD → corregir a 500 USD
inv_limpio.loc[inv_limpio["Costo_Unitario_USD"] == 0.05, "Costo_Unitario_USD"] = 500

# Smartphone con 850000 (COP) → convertir a USD con tasa 3,668.80 al día de hoy 01/02/2026
tasa_cop_usd = 3668.80
inv_limpio.loc[inv_limpio["Costo_Unitario_USD"] == 850000, "Costo_Unitario_USD"] = 850000 / tasa_cop_usd

### EXPORTACIÓN FINAL

In [ ]:
# Guardar el DataFrame como CSV
inv_limpio.to_csv("inventario_central_v2_limpio.csv", index=False, encoding="utf-8")

print("Archivo 'inventario_central_v2_limpio.csv' creado correctamente.")


Archivo 'inventario_central_v2_limpio.csv' creado correctamente.


## Auditoría de Calidad y Transparencia — Dataset Transacciones (trx)

In [ ]:
audit(trx)

,nulos_%,tipo
Transaccion_ID,0.00,object
SKU_ID,0.00,object
Fecha_Venta,0.00,object
Cantidad_Vendida,0.00,int64
Precio_Venta_Final,0.00,float64
Costo_Envio,8.34,float64
Tiempo_Entrega_Real,0.00,int64
Estado_Envio,16.83,object
Ciudad_Destino,0.00,object
Canal_Venta,0.00,object


### FUNCIONES DE LIMPIEZA Y NORMALIZACIÓN

In [ ]:
def normalize_text_full(s):
    if pd.isna(s): return s
    s = ftfy.fix_text(s)
    s = s.lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))
    s = re.sub(r"[^a-z0-9\s]", "", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def build_fuzzy_map(values, threshold=0.9):
    values = sorted(set(values.dropna()))
    canonical, mapping = [], {}
    for v in values:
        match = difflib.get_close_matches(v, canonical, n=1, cutoff=threshold)
        if match: mapping[v] = match[0]
        else:
            canonical.append(v)
            mapping[v] = v
    return mapping

def clean_numeric_outliers(df, numeric_cols, iqr_factor=1.5):
    df_clean = df.copy()
    for col in numeric_cols:
        series = df_clean[col]
        if series.dropna().empty: continue
        q1, q3 = series.quantile(0.25), series.quantile(0.75)
        iqr = q3 - q1
        lower, upper = q1 - iqr_factor * iqr, q3 + iqr_factor * iqr
        df_clean[col] = series.clip(lower, upper).fillna(series.median())
    return df_clean

### PROCESAMIENTO DE TRANSACCIONES

In [ ]:
trx["Cantidad_Vendida"] = trx["Cantidad_Vendida"].replace(-5, 5)
trx['Tiempo_Entrega_Real'] = trx['Tiempo_Entrega_Real'].replace(999, np.nan)

# Normalización de fechas
TODAY = pd.Timestamp("2026-01-31")
trx["Fecha_Venta"] = pd.to_datetime(trx["Fecha_Venta"], errors="coerce")
trx.loc[trx["Fecha_Venta"] > TODAY, "Fecha_Venta"] = pd.NaT

# Limpieza de strings vacíos o nulos disfrazados
EMPTY_VALUES = ["", " ", "nan", "NaN", "null", "NULL", "none", "None", "?", "-", "--"]
trx = trx.replace(EMPTY_VALUES, np.nan).replace(r"^\s*$", np.nan, regex=True)

/tmp/ipython-input-1510342991.py:6: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  trx["Fecha_Venta"] = pd.to_datetime(trx["Fecha_Venta"], errors="coerce")


### NORMALIZACIÓN DE TEXTO (Ciudades y Canales)

In [ ]:
city_aliases = {"med": "medellin", "mde": "medellin", "medell": "medellin", "bog": "bogota", "bta": "bogota", "bgta": "bogota"}

for col in ["Ciudad_Destino", "Canal_Venta"]:
    trx[f"{col}_norm"] = trx[col].apply(normalize_text_full)

trx["Ciudad_Destino_norm"] = trx["Ciudad_Destino_norm"].replace(city_aliases)

# Aplicar mapeo difuso (Fuzzy Matching)
city_map = build_fuzzy_map(trx["Ciudad_Destino_norm"], threshold=0.9)
trx["Ciudad_Destino_norm"] = trx["Ciudad_Destino_norm"].map(city_map)

### IMPUTACIÓN DE DATOS

In [ ]:
trx = trx[trx.isna().sum(axis=1) < 2].copy()

# Imputación Numérica Avanzada (Iterative Imputer)
numeric_cols = trx.select_dtypes(include="number").columns
it_imputer = IterativeImputer(max_iter=10, random_state=42)
trx[numeric_cols] = it_imputer.fit_transform(trx[numeric_cols])

# Winsorización de Outliers
trx = clean_numeric_outliers(trx, numeric_cols)

# Imputación Categórica (Random Forest para 'Estado_Envio')
col_obj = 'Estado_Envio'
cols_base = ['Cantidad_Vendida', 'Precio_Venta_Final', 'Costo_Envio', 'Tiempo_Entrega_Real']

if trx[col_obj].isna().any():
    imp_simple = SimpleImputer(strategy='median')
    X_val = pd.DataFrame(imp_simple.fit_transform(trx[cols_base]), index=trx.index)

    train_idx = trx[trx[col_obj].notna()].index
    predict_idx = trx[trx[col_obj].isna()].index

    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_val.loc[train_idx], trx.loc[train_idx, col_obj])

    trx.loc[predict_idx, col_obj] = rf_model.predict(X_val.loc[predict_idx])
    print(f"Imputados {len(predict_idx)} valores en {col_obj}")

Imputados 1516 valores en Estado_Envio


### EXPORTACIÓN FINAL

In [ ]:

trx.to_csv("transacciones_logistica_final_unificado.csv", index=False)
print("Proceso completado. Archivo guardado con éxito.")


Proceso completado. Archivo guardado con éxito.


## Auditoría de Calidad y Transparencia — Dataset Feedback (fb)

In [ ]:
audit(fb)

,nulos_%,tipo
Feedback_ID,0.000000,object
Transaccion_ID,0.000000,object
Rating_Producto,0.000000,int64
Rating_Logistica,0.000000,int64
Comentario_Texto,14.600000,object
Recomienda_Marca,24.866667,object
Ticket_Soporte_Abierto,0.000000,object
Edad_Cliente,0.000000,int64
Satisfaccion_NPS,0.000000,float64


In [6]:
fb

,Feedback_ID,Transaccion_ID,Rating_Producto,Rating_Logistica,Comentario_Texto,Recomienda_Marca,Ticket_Soporte_Abierto,Edad_Cliente,Satisfaccion_NPS
0,FB-8000,TRX-17461,99,4,NaN,NaN,Sí,195,-17.5
1,FB-8001,TRX-17755,4,5,---,Maybe,Sí,59,-41.7
2,FB-8002,TRX-10534,3,4,No volvería,Maybe,0,84,-36.4
3,FB-8003,TRX-12569,2,3,---,NaN,Sí,20,7.4
4,FB-8004,TRX-19159,4,2,Dañado,SI,No,83,61.0
...,...,...,...,...,...,...,...,...,...
4495,FB-11535,TRX-13156,3,1,Excelente,SI,1,54,-85.8
4496,FB-10167,TRX-14498,5,2,---,NO,Sí,70,80.2
4497,FB-8483,TRX-14656,2,5,Dañado,Maybe,Sí,66,28.2
4498,FB-10844,TRX-16982,4,4,Dañado,Maybe,0,27,-4.0


Se empieza revisando la columna de Ticket_Soporte_Abierto. Donde se evidencia valores binarios, por ello se decide cambiar los 1 por Si y los 0 por No, don el fin de manejar el mismo estandar.

In [7]:
fb['Ticket_Soporte_Abierto'].value_counts(dropna=False)

,count
Ticket_Soporte_Abierto,
Sí,1158
1,1140
0,1117
No,1085


In [8]:
fb['Ticket_Soporte_Abierto'] = (
    fb['Ticket_Soporte_Abierto']
    .astype(str)
    .str.strip()
    .replace({
        '1': 'Sí',
        '0': 'No',
        'Si': 'Sí',
        'Sí': 'Sí',
        'No': 'No'
    })
)


In [9]:
fb['Ticket_Soporte_Abierto'].value_counts(dropna=False)

,count
Ticket_Soporte_Abierto,
Sí,2298
No,2202


Luego se revisa la edad, para identificar valores mayores a 90 años. Se establece dicho valor como rango debido a que hay algunos usuarios que presentan edad superior a los 80 años.

In [10]:
fb[fb['Edad_Cliente'] > 90]

,Feedback_ID,Transaccion_ID,Rating_Producto,Rating_Logistica,Comentario_Texto,Recomienda_Marca,Ticket_Soporte_Abierto,Edad_Cliente,Satisfaccion_NPS
0,FB-8000,TRX-17461,99,4,NaN,NaN,Sí,195,-17.5
200,FB-8200,TRX-14110,2,3,Lento,SI,Sí,195,-64.9
400,FB-8400,TRX-16290,4,3,Lento,NO,No,195,3.8
600,FB-8600,TRX-19361,99,2,NaN,Maybe,Sí,195,-44.0
800,FB-8800,TRX-19439,1,3,Dañado,NO,No,195,1.6
1000,FB-9000,TRX-10621,5,1,Dañado,SI,No,195,-97.2
1200,FB-9200,TRX-13482,99,2,NaN,NaN,Sí,195,15.5
1400,FB-9400,TRX-15458,5,2,Dañado,SI,No,195,22.8
1600,FB-9600,TRX-19234,2,3,Precio justo,SI,Sí,195,80.4
1800,FB-9800,TRX-17185,99,3,Dañado,NaN,Sí,195,36.6


In [11]:
fb = fb[fb['Edad_Cliente'] <= 90]

Posterior a ello se revisa la columna Recomienda_Marca y se evidencia un valor Maybe, el cual se cambia a tal vez y valores invalidos los cuales se clasifican como sin respuesta.

In [12]:
fb['Recomienda_Marca'].value_counts(dropna=False)

,count
Recomienda_Marca,
SI,1156
NO,1137
NaN,1112
Maybe,1072


In [ ]:
fb['Recomienda_Marca'] = (
    fb['Recomienda_Marca']
    .astype(str)
    .str.strip()
    .replace({
        'SI': 'Sí',
        'NO': 'No',
        'Maybe': 'Tal vez',
        'nan': 'Sin respuesta'
    })
)

In [14]:
fb['Recomienda_Marca'].value_counts(dropna=False)


,count
Recomienda_Marca,
Sí,1156
No,1137
Sin respuesta,1112
Tal vez,1072


Al revisar la columna Satisfaccion_NPS se ven valores entre -100 y 100.Se verifica dicha informacion para no tener otros fuera de ese rango

In [15]:
fb[(fb['Satisfaccion_NPS'] > 100) | (fb['Satisfaccion_NPS'] < -100)]

,Feedback_ID,Transaccion_ID,Rating_Producto,Rating_Logistica,Comentario_Texto,Recomienda_Marca,Ticket_Soporte_Abierto,Edad_Cliente,Satisfaccion_NPS


In [ ]:
fb['Satisfaccion_NPS_norm'] = fb['Satisfaccion_NPS'] / 100


Se normallizan para continuar con la nomenclatura de NPS desde -100 hasta 100

In [ ]:
fb['Satisfaccion_NPS_norm'] = fb['Satisfaccion_NPS_norm'].round(2)

Luego se revisan losvalores en la colomna Comentario_texto y los valores NaN y --- se cambian por sin comentarios

In [18]:
fb['Comentario_Texto'].value_counts(dropna=False)

,count
Comentario_Texto,
Excelente,676
Lento,661
NaN,652
Dañado,643
---,629
No volvería,623
Precio justo,593


In [ ]:
fb['Comentario_Texto'] = (
    fb['Comentario_Texto']
    .replace('---', pd.NA)
    .fillna('Sin comentarios')
)


In [22]:
fb['Comentario_Texto'].value_counts(dropna=False)

,count
Comentario_Texto,
Sin comentarios,1281
Excelente,676
Lento,661
Dañado,643
No volvería,623
Precio justo,593


Se verifican valores Rating logistica y se evidenciia que estan dentro del limite de 1 a 5

In [23]:
fb['Rating_Logistica'].value_counts(dropna=False)

,count
Rating_Logistica,
1,915
3,914
5,905
4,895
2,848


Se hace lo mismo con rating prodcuto y se evidencian valores fuera del rango, se decide elimnarlos ya que elegir una un valorcon base en la calificación puede generar sesgo, ya que es subjetivo de la satisfacción de cada persona. para una pudo haber sido 3 para otra un 4. entonces se eliminan del dataset dichos valores


In [24]:
fb['Rating_Producto'].value_counts(dropna=False)

,count
Rating_Producto,
5,927
1,921
3,902
2,871
4,834
99,22


In [25]:
fb = fb[fb['Rating_Producto'].between(1, 5)]

In [26]:
fb['Rating_Producto'].value_counts(dropna=False)

,count
Rating_Producto,
5,927
1,921
3,902
2,871
4,834


Se evidencian Feedback_ID repetidos, debido a que varios productos se asocian con un feedback_ID se decide, mantener dichos valores de esta manera porque siguen arrojando información importante

In [27]:
fb['Feedback_ID'].duplicated().sum()

np.int64(489)

In [28]:
fb.groupby('Feedback_ID')['Transaccion_ID'].nunique().sort_values(ascending=False)

,Transaccion_ID
Feedback_ID,
FB-8151,4
FB-10102,3
FB-10929,3
FB-10925,3
FB-11447,3
...,...
FB-11440,1
FB-11441,1
FB-11442,1


Se genera una nueva columna, con la cantidad de Transacciones asociadas a cada Feedback y se guarda el dataset limpio

In [29]:
fb['Cantidad_Productos_Feedback'] = (
    fb.groupby('Feedback_ID')['Transaccion_ID']
    .transform('nunique')
)


In [30]:
fb.to_csv('feedback_clientes_limpio.csv', index=False, encoding='utf-8-sig')

In [31]:
fb

,Feedback_ID,Transaccion_ID,Rating_Producto,Rating_Logistica,Comentario_Texto,Recomienda_Marca,Ticket_Soporte_Abierto,Edad_Cliente,Satisfaccion_NPS,Satisfaccion_NPS_norm,Cantidad_Productos_Feedback
1,FB-8001,TRX-17755,4,5,Sin comentarios,Tal vez,Sí,59,-41.7,-0.42,1
2,FB-8002,TRX-10534,3,4,No volvería,Tal vez,No,84,-36.4,-0.36,1
3,FB-8003,TRX-12569,2,3,Sin comentarios,Sin respuesta,Sí,20,7.4,0.07,1
4,FB-8004,TRX-19159,4,2,Dañado,Sí,No,83,61.0,0.61,1
5,FB-8005,TRX-16051,5,3,Sin comentarios,Sí,Sí,77,95.0,0.95,1
...,...,...,...,...,...,...,...,...,...,...,...
4495,FB-11535,TRX-13156,3,1,Excelente,Sí,Sí,54,-85.8,-0.86,2
4496,FB-10167,TRX-14498,5,2,Sin comentarios,No,Sí,70,80.2,0.80,2
4497,FB-8483,TRX-14656,2,5,Dañado,Tal vez,Sí,66,28.2,0.28,3
4498,FB-10844,TRX-16982,4,4,Dañado,Tal vez,No,27,-4.0,-0.04,2
